Period based on mass of central object:
$$T = 2\pi\sqrt{\frac{r^{3}}{GM}}$$
Distance based on mass and period:
$$r = (GM(\frac{T}{2\pi})^{2})^{1/3}$$
Lorentz factor for velocity:
$$\gamma_{velocity}=\frac{1}{\sqrt{1-(\frac{v}{c})^{2}}}$$
Lorentz factor for gravitational acceleration
$$\gamma_{gravity}=\sqrt{1-\frac{2GM}{rc^{2}}}$$

In [15]:
# don't forget to upload this to git
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
%matplotlib notebook

# Setting Constants
G          = 6.674e-11 # N * m^2 * kg^-2
c          = 2.99e+8   # m * s^-1
sol_mass   = 1.99e+30  # kg
earth_mass = 5.97e+24  # kg
moon_mass  = 7.348e+22 # kg
moon_dist  = 3.84e+8   # m
jup_mass   = 1.89e+27  # kg
au         = 1.49e+11  # m
year       = 3600*24*365.25

# Initial Conditions
bh_mass    = 5e2* sol_mass
orb_mass   = 1# earth_mass
mu         = bh_mass * orb_mass


# Creating a function to calculate escape velocity
def calc_escape_velocity(mass, distance):
    V_esc = np.sqrt((2 * G * mass)/distance)
    return V_esc

# Creating a function to find the distances of chosen escape velocities
def calc_esc_radius(mass, V_esc):
    # Should give swarzchild radius when V_esc = c
    d = (2 * G * mass)/V_esc **2
    return d

def calc_radial_velocity(radius, period):
    rad_vel = (2 * np.pi * radius)/period
    return rad_vel

def calc_radius_from_vel(rad_vel, period):
    # r = vT/2*pi
    radius = (rad_vel * period)/ (2 * np.pi)
    return radius

def calc_period(distance, mass):
    period = 2 * np.pi *(math.sqrt(distance**3/(G * mass)))
    return period

def calc_grav_time_dilation(mass, distance):
    grav_gamma      = math.sqrt(1 - (2 * G * mass) / ((distance * c) ** 2))
    return grav_gamma

def calc_vel_time_dilation(velocity):
    if velocity >= 0.9999 * c:
        vel_gamma = 1/(math.sqrt(1-(0.9999/c)**2))
        return vel_gamma
    else:
        vel_gamma = 1/(math.sqrt(1-(velocity/c)**2))
        return vel_gamma
    
#moon_speed = calc_radial_velocity(moon_dist, 3600*24*27.32)
#print(f"Moon velocity = {moon_speed:.2f}m/s")

s_radius        = calc_esc_radius(bh_mass, c)
print(f"Schwarzschild Radius = {s_radius:.2f}m or {s_radius/au:.2e}au")

n = 1.51 # shouldn't go below 1.5
orbit_n         =  n * s_radius
print(f"{n} time(s) the event horizon radius: {orbit_n:.2e}m, or {orbit_n/1000:.2e}km, or {orbit_n/au:.2e}au")

bh_period       = calc_period(orbit_n, bh_mass)
print(f"Period at {n} * event horizon: {bh_period:.2e}s")

bh_orbit_speed  = calc_radial_velocity(orbit_n, bh_period)
if bh_orbit_speed >= c/10:
    print(f"Planet is orbiting with relativistic speeds at {bh_orbit_speed/c:.2f}c")
else:
    print(f"Speed of orbiting planet: {bh_orbit_speed:.2e}m/s or {bh_orbit_speed/c:.2f}c")
    
escape_v_n      = calc_escape_velocity(bh_mass, orbit_n)
print(f"Escape velocity at {n} schwarzchild radii: {escape_v_n:.2e}m/s or {escape_v_n/bh_orbit_speed:.2e} x radial speed or {escape_v_n/c:.2e}c.")

grav_gamma      = calc_grav_time_dilation(bh_mass, orbit_n)
print(f"Gravitational gamma factor {grav_gamma}")

vel_gamma       = calc_vel_time_dilation(escape_v_n)
print(f"Escape Velocity gamma factor {vel_gamma:.2f}")

radial_gamma    = calc_vel_time_dilation(bh_orbit_speed)
print(f"Radial orbit velocity gamma factor: {radial_gamma:.2f}")

# If the math is working right, the photon sphere should have an orbital speed of c
orbit_c         = calc_radial_velocity(1.5*s_radius, calc_period(1.5*s_radius,bh_mass))
if orbit_c == c:
    print("The photon sphere is accurately at 1.5 * the event horizon radius.")
else:
    print(f"The photon sphere is still broken - orbits are showing as stable at {orbit_c/c}c")

Schwarzschild Radius = 1485582.94m or 9.97e-06au
1.51 time(s) the event horizon radius: 2.24e+06m, or 2.24e+03km, or 1.51e-05au
Period at 1.51 * event horizon: 8.19e-02s
Planet is orbiting with relativistic speeds at 0.58c
Escape velocity at 1.51 schwarzchild radii: 2.43e+08m/s or 1.41e+00 x radial speed or 8.14e-01c.
Gravitational gamma factor 0.9999998523888242
Escape Velocity gamma factor 1.72
Radial orbit velocity gamma factor: 1.22
The photon sphere is still broken - orbits are showing as stable at 0.5773502691896257c


In [18]:
import pygame
import sys
import math
pygame.init()

# Creating a figure and axis
width, height = 700, 700
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Orbit Simulation")

# Color library
white        = (255,255,255)
black        = (0, 0, 0)
yellow       = (255, 255, 0)
blue         = (0, 0, 255)
red          = (255, 0, 0)
cosmic_latte = (255, 248, 231) # average color of galaxies
gray         = (150, 150, 150)

# Font
font         = pygame.font.SysFont("ariel", 16)

# Time and space scaling
# Still needs to handle whether or not a black hole is the central object
if bh_period >= year:
    dt       = year / 3600 
    scale    = 250 / au
else:
    dt       = bh_period/3600
    scale    = 250 / orbit_n

class Planet: 
    def __init__(self, x, y, radius, color, mass):
        self.x               = x
        self.y               = y
        self.radius          = radius # in pixels
        self.color           = color
        self.mass            = mass
        
        # Should make black hole object functionality
        # Should also make light object functionality
        self.sun             = False
        self.distance_to_sun = 1 # Avoiding division by 0
        self.orbit           = []
        
        self.bh              = False
        self.distance_to_bh  = 1 # Avoiding division by 0
        self.orbit           = []
        
        #if self.sun:  # Only calculate if it's not a sun object itself
         #   self.distance_to_sun, self.grav_gamma = self.calc_distance_to_center(sun)
        
        self.x_vel           = 0 # m/s
        self.y_vel           = 0 # m/s
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
        
        self.vel_gamma       = 1/(math.sqrt(1-(self.speed_magnitude/c)**2))
        
    #def calc_distance_to_center(self, other):
     #   if self.sun:
      #      return 1
       # elif self.bh:
        #    return 1
        #else:
         #   other_x, other_y = other.x, other.y
          #  distance_x       = other_x - self.x
           # distance_y       = other_y - self.y
            #distance         = math.sqrt(distance_x ** 2 + distance_y ** 2)
            #if other == self.bh:
             #   other_mass   = other.mass
              #  grav_gamma   = math.sqrt(1 - (2 * G * other_mass) / ((distance * c) ** 2))
            #if other == self.sun:
             #   other_mass   = other.mass
              #  grav_gamma   = math.sqrt(1 - (2 * G * other.mass) / ((distance * c) ** 2))
            
            #return distance, grav_gamma
        
    def draw(self, win):
        x = self.x * scale + width/2
        y = self.y * scale + height/2
        
        if len(self.orbit) > 2:
            updated_points = []
            for point in self.orbit:
                x, y = point
                x = x * scale + width/2
                y = y * scale + height/2
                updated_points.append((x,y))

            pygame.draw.lines(win, self.color, False, updated_points, 1)
        
            
        pygame.draw.circle(win, self.color, (x,y), self.radius)
        
        #if not self.sun:
         #   if self.distance_to_sun >= au/1000:
          #      distance_text = font.render(f"{self.distance_to_sun:.3e}au", 1, white)
           #     win.blit(distance_text, (x-distance_text.get_width()/2, y - distance_text.get_height()/2))
            #else:
             #   distance_text = font.render(f"{self.distance_to_sun:.3e}m", 1, white)
              #  win.blit(distance_text, (x-distance_text.get_width()/2, y - distance_text.get_height()/2))
        #if not self.sun:
         #   gamma_text = font.render(f"{grav_gamma}γ",1,white)
          #  win.blit(gamma_text, (x - gamma_text.get_width()/2, y - gamma_text.get_width()/2))
    
    #def draw_event_horizon(self, planets):
     #   for planet in planets:
      #      if planet == self and planet.bh:
       #         pygame.draw.circle(win, (255,255,255), (int(self.x), int(self.y)), int(s_radius), 1)

    
    def attraction(self, other):
        other_x, other_y = other.x, other.y
        distance_x       = other_x - self.x
        distance_y       = other_y - self.y
        distance         = math.sqrt(distance_x ** 2 + distance_y ** 2)
        
        if other.sun:
            self.distance_to_sun = distance
        elif other.bh:
            self.distance_to_sun = distance
        
        #mu      = self.mass * other.mass
        force   = G * self.mass * other.mass / distance ** 2
        theta   = math.atan2(distance_y, distance_x)
        force_x = math.cos(theta) * force
        force_y = math.sin(theta) * force
        return force_x, force_y
    
    def update_position(self, planets):
        # Using all planets, but I expect the central mass to cancel them all out.
        # I will not calculate the gamma factor affected by each planets
        total_fx = total_fy = 0
        for planet in planets:
            # Avoid division by 0
            if self == planet:
                continue
            fx, fy    = self.attraction(planet)
            total_fx += fx
            total_fy += fy
            
        # Update velocity
        self.x_vel += total_fx / self.mass * dt
        self.y_vel += total_fy / self.mass * dt
        
        # Update Position
        self.x     += self.x_vel * dt
        self.y     += self.y_vel * dt
        
        # Relative velocity
        self.x_vel *= self.vel_gamma #* self.grav_gamma
        self.y_vel *= self.vel_gamma #* self.grav_gamma
        
        self.orbit.append((self.x, self.y))
        
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
        if self.speed_magnitude >= c:
            # Cap velocity
            self.x_vel = self.x_vel / self.speed_magnitude * (c * 0.9999)
            self.y_vel = self.y_vel / self.speed_magnitude * (c * 0.9999)
             
def main():
    run            = True
    clock          = pygame.time.Clock()
    time_counter   = 0
    relative_time  = 0
    time_diff      = 0
    
    sun            = Planet(0,0, 9, yellow, sol_mass)
    sun.sun        = True
    
    black_hole     = Planet(0,0,3, white, bh_mass)
    black_hole.sun = True
    black_hole.bh  = True
    
    mercury        = Planet(0.39 * au, 0, 2, gray, 0.33e+24)
    mercury.y_vel  = -calc_radial_velocity(0.39*au, 0.24*year)
    
    venus          = Planet(0.72 * au, 0, 5, red, 4.87e+24)
    venus.y_vel    = -calc_radial_velocity(0.72*au, 0.62*year)
    
    earth          = Planet(au, 0, 2, blue, earth_mass)
    earth.y_vel    = -calc_radial_velocity(au, year)
    
    moon           = Planet(au + moon_dist, 0, 2, gray, moon_mass)
    moon.y_vel     = -(moon_speed-earth.y_vel)
    
    mars           = Planet(1.524 * au, 0, 4, red, 0.11 * earth_mass)
    mars.y_vel     = -2.41e4
    
    jupiter        = Planet(-5.2 * au, 0, 8, red, 0.1*sol_mass)#jup_mass)
    jupiter.y_vel  = calc_radial_velocity(5.2 * au, 12* year)
    
    saturn         = Planet(9.54 * au,0, 5, yellow, 568e+24)
    saturn.y_vel   = -calc_radial_velocity(9.54 * au, 28.7*year)
    
    t_plnt         = Planet(0.75 * orbit_n, 0, 2, white, 0.1*bh_mass)
    t_plnt.y_vel   = 1.25*bh_orbit_speed
    
    light          = Planet(1.5 * s_radius,0, 1, white, 1)
    light.y_vel    = -c#np.sin(c)
    #light.x_vel    = c#np.cos(c)
    
    t2_plnt        = Planet(1.3 * orbit_n, 0, 2, blue, 0.01 * bh_mass)
    t2_plnt.y_vel  = -0.5*escape_v_n
    
    planets = [black_hole, t_plnt]
    
    while run:
        clock.tick(60)
        win.fill(black)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
        
        # Update vel_gamma and grav_gamma counters
        vel_gamma           = calc_vel_time_dilation(math.sqrt(t_plnt.x_vel**2 + t_plnt.y_vel**2))
        # For some reason grav_gamma keeps spitting out imaginary numbers?
        grav_gamma          = (1 - (2 * float(G) * float(bh_mass))/(float(t_plnt.distance_to_sun) * float(c) **2))**0.5
        
        # Update time counter
        time_counter       += dt
        relative_time      += vel_gamma * dt
        time_diff           = relative_time - time_counter
        
        # Assign text boxes
        grav_gamma_text     = font.render(f"Time dilation factor from gravity = {grav_gamma:.3f}", 1, white)
        vel_gamma_text      = font.render(f"Time dilation factor from velocity = {vel_gamma:.3f}", 1, white)
        vel_text            = font.render(f"Velocity of planet: {t_plnt.speed_magnitude/c:.3f}c",1, white)
        if dt >= year/1000:
            time_text       = font.render(f"Inertial Clock: {time_counter/year:.3f}years", 1, white)  # Format time text
            rel_time_text   = font.render(f"Clock on planet: {relative_time/year:.3f}years", 1, white)
            dif_time_text   = font.render(f"Difference between inertial and relative time: {time_diff/year:.3f}years", 1, white)
        else:
            time_text       = font.render(f"Inertial Clock: {time_counter:.3f}s", 1, white)
            rel_time_text   = font.render(f"Clock on planet: {relative_time:.3f}s", 1, white)
            dif_time_text   = font.render(f"Difference between inertial and relative time: {time_diff:.3f}s", 1, white)
        
        
        # Print relevant values
        win.blit(time_text, (10, 10)) 
        win.blit(rel_time_text, (10, 30))
        win.blit(vel_gamma_text, (10, 50))
        win.blit(grav_gamma_text, (10, 70))
        win.blit(dif_time_text, (10, 90))
        win.blit(vel_text, (10, 110))
        
        for planet in planets:
            planet.update_position(planets)
            planet.draw(win)
            #if planet.bh:
             #   planet.draw_event_horizon(planets)

            
        pygame.display.update()
                
    pygame.quit
    
    
main()

SystemExit: 